In [ ]:
# Importações para Transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Hugging Face Transformers
from transformers import (
    AutoTokenizer, AutoModel, AutoModelForSequenceClassification,
    BertTokenizer, BertForSequenceClassification,
    RobertaTokenizer, RobertaForSequenceClassification,
    GPT2Tokenizer, GPT2LMHeadModel,
    pipeline, Trainer, TrainingArguments,
    DataCollatorWithPadding
)

# PyTorch
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Visualização
import plotly.express as px
import plotly.graph_objects as go

# Datasets
import sys
sys.path.append('..')
from datasets.textos_exemplo import *
from utils.nlp_utils import *

# Verificar GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Usando device: {device}")
print(f"🤗 Transformers versão: {transformers.__version__ if 'transformers' in globals() else 'Carregando...'}")
print("🚀 Transformers e modelos pré-treinados prontos!")


In [ ]:
# Primeiro, vamos usar pipelines pré-prontos
print("🚀 1. Usando Pipelines Pré-prontos")
print("=" * 40)

# Pipeline de análise de sentimentos
try:
    # Tentar usar modelo em português
    sentiment_pipeline = pipeline(
        "sentiment-analysis",
        model="neuralmind/bert-base-portuguese-cased",
        device=0 if torch.cuda.is_available() else -1
    )
    print("✅ Modelo BERT português carregado!")
except:
    # Fallback para modelo em inglês
    sentiment_pipeline = pipeline(
        "sentiment-analysis",
        model="cardiffnlp/twitter-roberta-base-sentiment-latest",
        device=0 if torch.cuda.is_available() else -1
    )
    print("✅ Modelo RoBERTa inglês carregado!")

# Testar com alguns textos
textos_teste = [
    "Adorei este produto, muito bom!",
    "Produto terrível, não recomendo.",
    "Qualidade média, poderia ser melhor.",
    "Excelente atendimento, muito satisfeito!",
    "Péssima experiência, muito decepcionante."
]

print(f"\n📝 Testando análise de sentimentos:")
for texto in textos_teste:
    resultado = sentiment_pipeline(texto)
    label = resultado[0]['label']
    score = resultado[0]['score']
    print(f"'{texto}'")
    print(f"  → {label} (confiança: {score:.3f})")
    print()

print("\n" + "="*60)
print("🤗 2. Usando Transformers Manualmente")
print("=" * 40)

# Carregar tokenizer e modelo manualmente
model_name = "distilbert-base-uncased"  # Modelo menor para demonstração
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=2  # Classificação binária
)

print(f"✅ Modelo {model_name} carregado!")
print(f"📊 Parâmetros do modelo: {model.num_parameters():,}")

# Preparar dados para fine-tuning
reviews_data = [
    ("Excelente produto, recomendo muito!", 1),
    ("Adorei a qualidade, superou expectativas.", 1),
    ("Muito bom, chegou rápido.", 1),
    ("Produto incrível, vale cada centavo!", 1),
    ("Estou muito satisfeito com a compra.", 1),
    ("Produto terrível, não recomendo.", 0),
    ("Muito ruim, dinheiro jogado fora.", 0),
    ("Qualidade péssima, chegou quebrado.", 0),
    ("Horrível, não serve para nada.", 0),
    ("Decepcionante, esperava muito mais.", 0),
]

texts = [item[0] for item in reviews_data]
labels = [item[1] for item in reviews_data]

print(f"\n📊 Dataset preparado:")
print(f"Total de exemplos: {len(texts)}")
print(f"Positivos: {sum(labels)}")
print(f"Negativos: {len(labels) - sum(labels)}")

# Tokenizar textos
print(f"\n🔤 Tokenizando textos...")
encoded = tokenizer(
    texts,
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors='pt'
)

print(f"✅ Tokenização completa!")
print(f"📊 Forma dos input_ids: {encoded['input_ids'].shape}")
print(f"📊 Forma da attention_mask: {encoded['attention_mask'].shape}")

# Exemplo de tokenização
print(f"\n📝 Exemplo de tokenização:")
exemplo_texto = texts[0]
exemplo_tokens = tokenizer.tokenize(exemplo_texto)
exemplo_ids = tokenizer.convert_tokens_to_ids(exemplo_tokens)

print(f"Texto: '{exemplo_texto}'")
print(f"Tokens: {exemplo_tokens}")
print(f"IDs: {exemplo_ids}")

# Fazer predição com modelo pré-treinado (sem fine-tuning)
print(f"\n🔮 Predições com modelo pré-treinado:")
model.eval()
with torch.no_grad():
    outputs = model(**encoded)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_labels = torch.argmax(predictions, dim=-1)

for i, (texto, pred_label, probs) in enumerate(zip(texts[:5], predicted_labels[:5], predictions[:5])):
    real_label = labels[i]
    confidence = torch.max(probs).item()
    
    print(f"'{texto[:50]}...'")
    print(f"  Real: {real_label}, Predito: {pred_label.item()}, Confiança: {confidence:.3f}")
    print()

print("\n" + "="*60)
print("🎯 3. Comparando com Métodos Tradicionais")
print("=" * 40)

# Comparar com TF-IDF + Logistic Regression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Preparar dados para sklearn
X_traditional = texts
y_traditional = labels

# TF-IDF + Logistic Regression
tfidf = TfidfVectorizer(max_features=1000)
X_tfidf = tfidf.fit_transform(X_traditional)

lr_model = LogisticRegression(random_state=42)
lr_scores = cross_val_score(lr_model, X_tfidf, y_traditional, cv=3, scoring='accuracy')

print(f"📊 Comparação de Performance:")
print(f"TF-IDF + Logistic Regression: {lr_scores.mean():.3f} (±{lr_scores.std():.3f})")

# Calcular accuracy do modelo pré-treinado
transformer_accuracy = accuracy_score(labels, predicted_labels.numpy())
print(f"BERT pré-treinado (sem fine-tuning): {transformer_accuracy:.3f}")

print(f"\n💡 Observações:")
print(f"- O modelo BERT foi treinado em dados gerais, não específicos para reviews")
print(f"- Com fine-tuning, a performance do BERT seria significativamente melhor")
print(f"- Para datasets pequenos, métodos tradicionais podem ser mais eficientes")

# Demonstrar capacidades do modelo
print(f"\n🧪 Testando capacidades do modelo:")
novos_textos = [
    "Este produto mudou minha vida!",
    "Pior compra que já fiz na vida.",
    "Produto ok, nada demais.",
    "Simplesmente fantástico, recomendo!"
]

# Tokenizar novos textos
new_encoded = tokenizer(
    novos_textos,
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors='pt'
)

# Predizer
with torch.no_grad():
    new_outputs = model(**new_encoded)
    new_predictions = torch.nn.functional.softmax(new_outputs.logits, dim=-1)
    new_labels = torch.argmax(new_predictions, dim=-1)

print(f"\n🔮 Predições para novos textos:")
for texto, pred, probs in zip(novos_textos, new_labels, new_predictions):
    sentiment = "Positivo" if pred.item() == 1 else "Negativo"
    confidence = torch.max(probs).item()
    
    print(f"'{texto}'")
    print(f"  → {sentiment} (confiança: {confidence:.3f})")
    print()
